In [1]:
from bsm import bs_price, greeks, implied_vol_newton

S, K = 200.0, 210.0
T = 30/365
r, q = 0.05, 0.00
sigma = 0.20

price = bs_price(S, K, T, r, sigma, "call", q)
g = greeks(S, K, T, r, sigma, "call", q)
price, g

(np.float64(1.4615811587707057),
 {'delta': np.float64(0.2264536760489173),
  'gamma': np.float64(0.02624838509120609),
  'vega': np.float64(17.25921211476565),
  'theta_per_year': np.float64(-23.19016577551551),
  'rho': np.float64(3.602396223370911)})

In [2]:
true_sigma = 0.30
market_px = bs_price(S, K, T, r, true_sigma, "call", q)
iv = implied_vol_newton(market_px, S, K, T, r, "call", q)
true_sigma, market_px, iv

(0.3, np.float64(3.3730051848583997), np.float64(0.3000000010254372))

In [3]:
import yfinance as yf
import pandas as pd

tkr = yf.Ticker("AAPL")
hist = tkr.history(period="5d", interval="1d")
S_aapl = float(hist["Close"].iloc[-1])

divs = tkr.dividends
one_year_ago = pd.Timestamp.utcnow().tz_localize(None) - pd.Timedelta(days=365)
divs_last_year = divs[divs.index.tz_localize(None) >= one_year_ago]
div_sum = float(divs_last_year.sum()) if not divs_last_year.empty else 0.0
q_aapl = div_sum / S_aapl if S_aapl > 0 else 0.0

S_aapl, q_aapl

(239.69000244140625, 0.004255496639870683)

In [4]:
S = S_aapl
K = round(S_aapl, 0)  # near the money
T = 30/365
r = 0.05
q = q_aapl
sigma = 0.25

px = bs_price(S, K, T, r, sigma, "put", q)
greeks(S, K, T, r, sigma, "put", q), px

({'delta': np.float64(-0.47182777377272944),
  'gamma': np.float64(0.023156987415316785),
  'vega': np.float64(27.33696509349413),
  'theta_per_year': np.float64(-36.07410969370893),
  'rho': np.float64(-9.833625833251915)},
 np.float64(6.5500473903895085))

In [5]:
exps = tkr.options
expiry = exps[0]  # nearest weekly
chain = tkr.option_chain(expiry)
calls = chain.calls[["strike", "bid", "ask", "lastPrice", "impliedVolatility"]].dropna()

calls["dist"] = (calls["strike"] - S).abs()
row = calls.sort_values("dist").iloc[0]

strike_mtm = float(row["strike"])
mid = float((row["bid"] + row["ask"]) / 2.0) if row["bid"] > 0 and row["ask"] > 0 else float(row["lastPrice"])
iv_yf = float(row["impliedVolatility"])

expiry_dt = pd.to_datetime(expiry).tz_localize("UTC")
today = pd.Timestamp.utcnow().normalize()
T_chain = max((expiry_dt - today).days, 1) / 365.0

iv_me = implied_vol_newton(mid, S, strike_mtm, T_chain, r, "call", q)
{
    "S": S, "K": strike_mtm, "market_mid": mid,
    "T_years": T_chain, "iv_yfinance": iv_yf, "iv_me": iv_me
}

{'S': 239.69000244140625,
 'K': 240.0,
 'market_mid': 2.755,
 'T_years': 0.0136986301369863,
 'iv_yfinance': 0.23987576538085933,
 'iv_me': np.float64(0.253171078361114)}

In [6]:

from datetime import date

tkr = yf.Ticker("AAPL")
exps = tkr.options
expiry = date(2025, 8, 15)
print(expiry)
print(type(expiry))
# chain = tkr.option_chain(expiry)
# calls = chain.calls[["strike","bid","ask","lastPrice","impliedVolatility"]].dropna().copy()

2025-08-15
<class 'datetime.date'>


In [7]:
exps = tkr.options
expiry = exps[2]
chain = tkr.option_chain(expiry)
calls = chain.calls[["strike", "bid", "ask", "lastPrice", "impliedVolatility"]].dropna().copy()

In [8]:
exps = tkr.options
expiry = exps[2]
chain = tkr.option_chain(expiry)
calls = chain.calls[["strike", "bid", "ask", "lastPrice", "impliedVolatility"]].dropna().copy()

In [13]:
df = pd.read_excel(
    "data/options_latest.xlsx",
    header=[0, 1]
)
print(df.columns.to_list())

[('CALLS', 'OI'), ('CALLS', 'CHNG IN OI'), ('CALLS', 'VOLUME'), ('CALLS', 'IV'), ('CALLS', 'LTP'), ('CALLS', 'CHNG'), ('CALLS', 'BID QTY'), ('CALLS', 'BID'), ('CALLS', 'ASK'), ('CALLS', 'ASK QTY'), ('STRIKE', 'STRIKE'), ('PUTS', 'BID QTY'), ('PUTS', 'BID'), ('PUTS', 'ASK'), ('PUTS', 'ASK QTY'), ('PUTS', 'CHNG'), ('PUTS', 'LTP'), ('PUTS', 'IV'), ('PUTS', 'VOLUME'), ('PUTS', 'CHNG IN OI'), ('PUTS', 'OI')]


In [14]:
df = df.set_index(("STRIKE", "STRIKE"))

In [15]:
df.head()

CALLS                                                     \
                    OI CHNG IN OI VOLUME IV     LTP  CHNG BID QTY     BID   
(STRIKE, STRIKE)                                                            
1140                 3          1      1  -     234  -0.4     500  233.85   
1160                 9          -      -  -  207.15     -    2500  212.75   
1180              1180          -      7  -     190  -9.2     500  197.10   
1200               141         -3      4  -     172 -1.25     500  177.25   
1220               189          -      -  -  143.05     -   10000  153.75   

                                    PUTS                                  \
                     ASK ASK QTY BID QTY   BID   ASK ASK QTY  CHNG   LTP   
(STRIKE, STRIKE)                                                           
1140              241.60     500   21000  0.35   0.4    7000 -0.05   0.4   
1160              225.95    1500    1000   0.4  0.45   11500     -   0.4   
1180              199.20     500    2000   0.4  0.45   22500  -0.1   0.4   
1200              179.25     500   16500  0.55   0.6    5000 -0.25  0.55   
1220              167.00   10000   33500  0.65  0.75    3000 -0.35   0.7   

                                                 
                     IV VOLUME CHNG IN OI    OI  
(STRIKE, STRIKE)                                 
1140               32.5     60         20   210  
1160              29.93     27        -20   173  
1180              27.38     34          -   357  
1200                 26   1808       -145  3023  
1220              24.28     74          1   471

In [16]:
calls_df = df['CALLS']
calls_df.head()

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
"(STRIKE, STRIKE)",,,,,,,,,,
1140,3,1,1,-,234,-0.4,500,233.85,241.60,500
1160,9,-,-,-,207.15,-,2500,212.75,225.95,1500
1180,1180,-,7,-,190,-9.2,500,197.10,199.20,500
1200,141,-3,4,-,172,-1.25,500,177.25,179.25,500
1220,189,-,-,-,143.05,-,10000,153.75,167.00,10000


In [17]:
print(calls_df.columns.to_list())

['OI', 'CHNG IN OI', 'VOLUME', 'IV', 'LTP', 'CHNG', 'BID QTY', 'BID', 'ASK', 'ASK QTY']


In [18]:
calls_df = calls_df.reset_index()

calls_df.head()

,"(STRIKE, STRIKE)",OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
0,1140,3,1,1,-,234,-0.4,500,233.85,241.60,500
1,1160,9,-,-,-,207.15,-,2500,212.75,225.95,1500
2,1180,1180,-,7,-,190,-9.2,500,197.10,199.20,500
3,1200,141,-3,4,-,172,-1.25,500,177.25,179.25,500
4,1220,189,-,-,-,143.05,-,10000,153.75,167.00,10000


In [19]:
calls_df = calls_df.rename(columns={ calls_df.columns[0]: "STRIKE" })
print(calls_df.columns.to_list())

['STRIKE', 'OI', 'CHNG IN OI', 'VOLUME', 'IV', 'LTP', 'CHNG', 'BID QTY', 'BID', 'ASK', 'ASK QTY']


In [20]:
calls_df.head()

,STRIKE,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
0,1140,3,1,1,-,234,-0.4,500,233.85,241.60,500
1,1160,9,-,-,-,207.15,-,2500,212.75,225.95,1500
2,1180,1180,-,7,-,190,-9.2,500,197.10,199.20,500
3,1200,141,-3,4,-,172,-1.25,500,177.25,179.25,500
4,1220,189,-,-,-,143.05,-,10000,153.75,167.00,10000


In [21]:
calls_df.columns = ['_'.join(map(str, col)).strip().lower().replace(' ', '_') 
              if isinstance(col, tuple) else col.lower().replace(' ', '_') 
              for col in calls_df.columns]

print(calls_df.columns.to_list())

['strike', 'oi', 'chng_in_oi', 'volume', 'iv', 'ltp', 'chng', 'bid_qty', 'bid', 'ask', 'ask_qty']
